## MLflow Registry: Gemini (Google GenAI) — Minimal Notebook

**Objetivo:** Validar *solo* la parte del **Model Registry** de MLflow con un modelo Chat que usa **Google GenAI (Gemini)**.

In [45]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [46]:
import os
gemini_api_key = os.getenv('GEMINI_API_KEY')


In [47]:
import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("tracking_llm_real")
print("Tracking URI:", mlflow.get_tracking_uri())

Tracking URI: http://127.0.0.1:5000


## Creación del experimento.

In [48]:
import mlflow
import time
import json
from mlflow.tracking import MlflowClient

# Crear o usar un experimento para LLMs
mlflow.set_experiment("llm_genai_model_registry")

client = MlflowClient()



# Google model 

In [49]:
from openai import OpenAI

gemini = OpenAI(api_key=gemini_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
model_name = "gemini-2.0-flash"

response = gemini.chat.completions.create(model=model_name, messages=[
    {
      "role": "user",
      "content": "Que es AI"
    }
  ])
answer = response.choices[0].message.content
answer

'AI son las siglas en inglés de **Artificial Intelligence**, que en español se traduce como **Inteligencia Artificial (IA)**.\n\nEn términos generales, la IA se refiere a la **capacidad de una máquina o sistema informático para imitar las capacidades cognitivas humanas**, como el aprendizaje, el razonamiento, la resolución de problemas, la percepción (visual y auditiva), la comprensión del lenguaje natural y la toma de decisiones.\n\nEs importante destacar que la IA no es una sola cosa, sino un campo amplio y en constante evolución que engloba diferentes técnicas y enfoques.\n\nAquí te dejo una desglose de los aspectos clave de la IA:\n\n*   **Objetivo:** Crear máquinas que puedan realizar tareas que normalmente requieren inteligencia humana.\n*   **Técnicas:** Se basa en una variedad de técnicas, incluyendo el aprendizaje automático (machine learning), el aprendizaje profundo (deep learning), el procesamiento del lenguaje natural (NLP), la visión artificial (computer vision), la robót

In [50]:
import os, time
from openai import OpenAI

# Config Gemini usando el endpoint OpenAI-compatible
gemini = OpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)
model_name = "gemini-2.0-flash"

prompt = "¿Qué es la inteligencia artificial?"

with mlflow.start_run(run_name="gemini_tracking"):
    # Registrar parámetros
    mlflow.log_params({
        "provider": "gemini",
        "model_name": model_name,
        "temperature": 0.0
    })

    # Medir latencia
    t0 = time.perf_counter()
    response = gemini.chat.completions.create(
        model=model_name,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0,
    )
    latency_ms = (time.perf_counter() - t0) * 1000

    answer = response.choices[0].message.content

    # Registrar métricas
    mlflow.log_metric("latency_ms", latency_ms)
    mlflow.log_metric("prompt_tokens", getattr(response.usage, "prompt_tokens", 0))
    mlflow.log_metric("completion_tokens", getattr(response.usage, "completion_tokens", 0))

    # Guardar la salida como artefacto
    with open("gemini_output.txt", "w") as f:
        f.write(f"Prompt: {prompt}\n\nRespuesta: {answer}")
    mlflow.log_artifact("gemini_output.txt")

print("✅ GEMINI:", answer)


🏃 View run gemini_tracking at: http://127.0.0.1:5000/#/experiments/539662435075150292/runs/d81a92db08e4437f811e9231e7e83dbd
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/539662435075150292
✅ GEMINI: La Inteligencia Artificial (IA) es un campo de la informática que se centra en la creación de sistemas informáticos capaces de realizar tareas que normalmente requieren inteligencia humana. En otras palabras, busca **simular la inteligencia humana en máquinas**.

Aquí te desglosamos los aspectos clave de la IA:

*   **Simulación de la Inteligencia Humana:** La IA busca replicar o imitar habilidades cognitivas humanas como:
    *   **Aprendizaje:** La capacidad de aprender de los datos y mejorar el rendimiento con la experiencia.
    *   **Razonamiento:** La capacidad de resolver problemas, tomar decisiones y sacar conclusiones lógicas.
    *   **Percepción:** La capacidad de interpretar y comprender información sensorial (como imágenes, sonido, texto).
    *   **Resolución de Pr

## Tracker de Gemini

In [51]:
def generate_gemini(prompt: str, temperature: float = 0.2):
    t0 = time.time()
    response = gemini.chat.completions.create(
        model="gemini-2.0-flash",
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature
    )
    latency = time.time() - t0
    text = response.choices[0].message.content
    tokens = len((prompt + " " + text).split())  # simulación simple de tokens
    return {"response": text, "latency": latency, "tokens": tokens}


# Ollama 

In [52]:
import time
from openai import OpenAI

# Cliente OpenAI apuntando al endpoint local de Ollama
ollama = OpenAI(
    api_key="ollama",  # obligatorio, aunque Ollama no lo usa
    base_url="http://localhost:11434/v1"
)
model_name = "llama3.2"
prompt = "Explica qué es MLflow en una frase."

with mlflow.start_run(run_name="ollama_tracking"):
    mlflow.log_params({
        "provider": "ollama",
        "model_name": model_name,
        "temperature": 0.0,
        
    })

    t0 = time.perf_counter()
    response = ollama.chat.completions.create(
        model=model_name,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0,
    )
    latency_ms = (time.perf_counter() - t0) * 1000

    answer = response.choices[0].message.content

    mlflow.log_metric("latency_ms", latency_ms)

    with open("ollama_output.txt", "w") as f:
        f.write(f"Prompt: {prompt}\n\nRespuesta: {answer}")
    mlflow.log_artifact("ollama_output.txt")

print("✅ OLLAMA:", answer)

🏃 View run ollama_tracking at: http://127.0.0.1:5000/#/experiments/539662435075150292/runs/c3f2955be69446aa9dc018055bd375bd
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/539662435075150292
✅ OLLAMA: MLflow es una plataforma de ciencia de datos y aprendizaje automático que permite a los desarrolladores y científicos de datos crear, entrenar y depurar modelos de machine learning de manera eficiente y escalable.


## Tracker de Ollama

In [53]:
def generate_ollama(prompt: str, temperature: float = 0.0):
    t0 = time.time()
    response = ollama.chat.completions.create(
        model="llama3.2",
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature
    )
    latency = time.time() - t0
    text = response.choices[0].message.content
    tokens = len((prompt + " " + text).split())
    return {"response": text, "latency": latency, "tokens": tokens}


## Registro de ejecuciones de LLMs

In [54]:
def track_llm(provider, model_name, fn, prompt, temperature=0.2, cost_per_token=0.000002):
    with mlflow.start_run(run_name=f"{provider}_run"):
        result = fn(prompt, temperature)
        
        # Log params
        mlflow.log_param("provider", provider)
        mlflow.log_param("model_name", model_name)
        mlflow.log_param("temperature", temperature)
        mlflow.log_param("task", "chat")

        # Log metrics
        mlflow.log_metric("latency_s", result["latency"])
        mlflow.log_metric("total_tokens", result["tokens"])
        mlflow.log_metric("estimated_cost", result["tokens"] * cost_per_token)

        # Log artifacts (prompt y respuesta)
        with open("prompt.txt", "w") as f:
            f.write(prompt)
        with open("response.txt", "w") as f:
            f.write(result["response"])

        mlflow.log_artifact("prompt.txt", artifact_path="prompts")
        mlflow.log_artifact("response.txt", artifact_path="responses")

        print(f"{provider} registrado en MLflow ({model_name})")
        return result


## Ejecución de ambos modelos.

In [55]:
prompt = "Explica brevemente qué hace MLflow y por qué es útil para modelos de lenguaje."

gemini_result = track_llm("gemini", "llm_gemini_chat", generate_gemini, prompt, temperature=0.2)
ollama_result = track_llm("ollama", "llm_ollama_chat", generate_ollama, prompt, temperature=0.0)


gemini registrado en MLflow (llm_gemini_chat)
🏃 View run gemini_run at: http://127.0.0.1:5000/#/experiments/539662435075150292/runs/0e39ccf1ddb64f7cb658d4fb9d474f1b
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/539662435075150292
ollama registrado en MLflow (llm_ollama_chat)
🏃 View run ollama_run at: http://127.0.0.1:5000/#/experiments/539662435075150292/runs/9c7090e1d2f74d66813c3ac895b9c830
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/539662435075150292


## Registro de ambos modelos en el model registry

In [56]:
import os
import mlflow
import mlflow.pyfunc

# Configurar el tracking local en la carpeta mlruns
mlflow.set_tracking_uri("file://" + os.path.abspath("mlruns"))

# Crear (o usar) el experimento
mlflow.set_experiment("genai_model_registry")

# Crear un modelo simulado compatible con PyFunc
class DummyModel(mlflow.pyfunc.PythonModel):
    def predict(self, context, model_input):
        return ["respuesta generada"]

# --- Registrar GEMINI ---
with mlflow.start_run(run_name="registro_llm_gemini") as run_gemini:
    mlflow.pyfunc.log_model(
        artifact_path="model",
        python_model=DummyModel()
    )
    model_uri_gemini = f"runs:/{run_gemini.info.run_id}/model"
    mlflow.register_model(model_uri=model_uri_gemini, name="llm_gemini_chat")
    print("Modelo 'llm_gemini_chat' registrado correctamente.")

# --- Registrar OLLAMA ---
with mlflow.start_run(run_name="registro_llm_ollama") as run_ollama:
    mlflow.pyfunc.log_model(
        artifact_path="model",
        python_model=DummyModel()
    )
    model_uri_ollama = f"runs:/{run_ollama.info.run_id}/model"
    mlflow.register_model(model_uri=model_uri_ollama, name="llm_ollama_chat")
    print("Modelo 'llm_ollama_chat' registrado correctamente.")





/usr/local/lib/python3.12/site-packages/mlflow/pyfunc/utils/data_validation.py:186: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(
2025/10/29 13:20:46 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


2025/10/29 13:20:51 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'llm_gemini_chat' already exists. Creating a new version of this model...
2025/10/29 13:20:51 WARNING mlflow.tracking._model_registry.fluent: Run with id be86ea47b8e040e8a6823e88067faa68 has no artifacts at artifact path 'model', registering model based on models:/m-4d24717049574df1b6b07589c51b2a6a instead
Created version '3' of model 'llm_gemini_chat'.
2025/10/29 13:20:51 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Modelo 'llm_gemini_chat' registrado correctamente.


2025/10/29 13:20:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'llm_ollama_chat' already exists. Creating a new version of this model...
2025/10/29 13:20:55 WARNING mlflow.tracking._model_registry.fluent: Run with id ca6fb8e9047f4afa845f83e6bedda8e9 has no artifacts at artifact path 'model', registering model based on models:/m-372020f553c24026a0b132d20b634576 instead


Modelo 'llm_ollama_chat' registrado correctamente.


Created version '2' of model 'llm_ollama_chat'.


## Promovemos los modelos a Stagging

In [57]:
from mlflow import MlflowClient

# Crear cliente
client = MlflowClient()

# Lista de modelos a promover
model_names = ["llm_gemini_chat", "llm_ollama_chat"]

for name in model_names:
    try:
        versions = client.get_latest_versions(name)
        if versions:
            version = versions[0].version
            client.transition_model_version_stage(
                name=name,
                version=version,
                stage="Staging",
                archive_existing_versions=True
            )
            print(f"✅ {name} v{version} promovido a Staging")
        else:
            print(f"⚠️ No se encontraron versiones para {name}")
    except Exception as e:
        print(f"❌ Error al promover {name}: {e}")


/var/folders/69/gxwhgd3s2v73lfrd296fzzs40000gn/T/ipykernel_74397/3515754190.py:11: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  versions = client.get_latest_versions(name)
/var/folders/69/gxwhgd3s2v73lfrd296fzzs40000gn/T/ipykernel_74397/3515754190.py:14: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


✅ llm_gemini_chat v1 promovido a Staging
✅ llm_ollama_chat v2 promovido a Staging


## Comparación de métricas entre modelos.

In [58]:
import mlflow
import pandas as pd
from mlflow import MlflowClient
import os

mlflow.set_tracking_uri("file://" + os.path.abspath("mlruns")) # Usando la de archivos como en tu notebook
client = MlflowClient()

# 1. APUNTAR AL EXPERIMENTO CORRECTO
experiment = mlflow.get_experiment_by_name("llm_genai_model_registry")

# Obtener todos los runs del experimento
runs_df = mlflow.search_runs(experiment_ids=[experiment.experiment_id])

# 2. USAR LOS NOMBRES CORRECTOS DE LAS COLUMNAS
# Estos son los nombres reales que registraste con track_llm
cols = [
    "tags.mlflow.runName",
    "params.provider",
    "params.model_name",
    "params.temperature",
    "metrics.latency_s",
    "metrics.total_tokens",
    "metrics.estimated_cost"
]

# Filtrar y renombrar columnas
existing_cols = [c for c in cols if c in runs_df.columns]
comparison = runs_df[existing_cols].rename(columns={
    "tags.mlflow.runName": "Ejecución",
    "params.provider": "Proveedor",
    "params.model_name": "Modelo",
    "params.temperature": "Temperatura",
    "metrics.latency_s": "Latencia (s)",
    "metrics.total_tokens": "Tokens totales",
    "metrics.estimated_cost": "Costo estimado ($)"
})

# Mostrar comparación
print("📊 Comparación de modelos Gemini vs Ollama:")
display(comparison)

# Mostrar resumen comparativo promedio
if not comparison.empty and "Proveedor" in comparison.columns:
    resumen = comparison.groupby("Proveedor")[["Latencia (s)", "Costo estimado ($)"]].mean().round(4)
    print("\n📈 Promedio de métricas por proveedor:")
    display(resumen)
else:
    print("⚠️ No se pudieron generar las métricas. Revisa los nombres del experimento y las columnas.")





📊 Comparación de modelos Gemini vs Ollama:


,Ejecución,Proveedor,Modelo,Temperatura,Latencia (s),Tokens totales,Costo estimado ($)
0,ollama_run,ollama,llm_ollama_chat,0.0,47.257168,225.0,0.000450
1,gemini_run,gemini,llm_gemini_chat,0.2,13.125844,291.0,0.000582
2,ollama_tracking,ollama,llama3.2,0.0,NaN,NaN,NaN
3,gemini_tracking,gemini,gemini-2.0-flash,0.0,NaN,NaN,NaN
4,ollama_run,ollama,llm_ollama_chat,0.0,51.061068,225.0,0.000450
5,gemini_run,gemini,llm_gemini_chat,0.2,4.600256,337.0,0.000674
6,ollama_tracking,ollama,llama3.2,0.0,NaN,NaN,NaN
7,gemini_tracking,gemini,gemini-2.0-flash,0.0,NaN,NaN,NaN



📈 Promedio de métricas por proveedor:


,Latencia (s),Costo estimado ($)
Proveedor,,
gemini,8.8630,0.0006
ollama,49.1591,0.0004
